In [1]:
import tensorflow as tf
import numpy as np
import cv2 as cv
import json, random

In [2]:
IMAGE_DIM = (380, 380, 3)
label_map = []
dataset, labels = [], []
raw_file = []

def get_category_index(cat_list):
    cat_ind_list = []
    cat_list = cat_list[1:3]
    for cat in cat_list:
        if not cat in label_map:
            label_map.append(cat)
        cat_ind_list.append(label_map.index(cat))    
    return cat_ind_list
with open('dataset/dataset.txt') as f:
    raw_file = f.readlines()[:1000]
#     random.shuffle(raw_file)
for record in raw_file:
    data = json.loads(record)
    image = cv.imread('dataset/'+data['dir'])
    if not image.shape == IMAGE_DIM:
        continue
    label = get_category_index(data['label'])
    dataset.append(image)
    labels.append(label)
for i in range(len(labels)):
    local_label = np.zeros(len(label_map))
    local_label[labels[i]] = 1
    labels[i] = local_label
    
dataset = np.array(dataset)
labels = np.asarray(labels)

print 'Images: ', dataset.shape
print 'Labels: ', labels.shape
print 'Categories: ', len(label_map)

Images:  (994, 380, 380, 3)
Labels:  (994, 41)
Categories:  41


In [3]:
SPLIT_WEIGHTS = (8, 1, 1) # train, validation, test
num_train, num_val, num_test = ( len(dataset)*weight/10 for weight in SPLIT_WEIGHTS )
train_set, val_set, test_set = np.split(dataset, [num_train, num_train+num_val])
train_label, val_label, test_label = np.split(labels, [num_train, num_train+num_val])
print "Training: ", len(train_set), "Validation: ", len(val_set), "Test: ", len(test_set)

Training:  795 Validation:  99 Test:  100


In [4]:
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=IMAGE_DIM,
    pooling=None
)
base_model.trainable = False
feature_set = base_model(tf.cast(dataset, tf.float32))
print "Feature set: ", feature_set.shape

/Users/coviam/image_search/pyenv/lib/python2.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Feature set:  (994, 12, 12, 2048)


In [5]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_set)
print "Averaging layer: ", feature_batch_average.shape

Averaging layer:  (994, 2048)


In [6]:
prediction_layer = tf.keras.layers.Dense(len(label_map), activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print "Prediction set: ",  prediction_batch.shape

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Prediction set:  (994, 41)


In [7]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])


In [8]:
alpha = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=alpha),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 12, 12, 2048)      23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 41)                84009     
Total params: 23,671,721
Trainable params: 84,009
Non-trainable params: 23,587,712
_________________________________________________________________


In [20]:
# validation_steps = 20

# loss0, accuracy0 = model.evaluate(val_set, val_label, steps = validation_steps)

KeyboardInterrupt: 

In [11]:
epochs = 5
history = model.fit(train_set, train_label,
         epochs=epochs,
         validation_data = (val_set, val_label))

Train on 795 samples, validate on 99 samples
Epoch 1/5
795/795 [==============================] - 703s 884ms/sample - loss: 0.1779 - acc: 0.9512 - val_loss: 0.1783 - val_acc: 0.9512
Epoch 2/5
795/795 [==============================] - 670s 842ms/sample - loss: 0.1506 - acc: 0.9513 - val_loss: 0.1627 - val_acc: 0.9512
Epoch 3/5
795/795 [==============================] - 706s 889ms/sample - loss: 0.1348 - acc: 0.9516 - val_loss: 0.1508 - val_acc: 0.9512
Epoch 4/5
795/795 [==============================] - 734s 924ms/sample - loss: 0.1211 - acc: 0.9520 - val_loss: 0.1425 - val_acc: 0.9510
Epoch 5/5
795/795 [==============================] - 745s 937ms/sample - loss: 0.1099 - acc: 0.9526 - val_loss: 0.1325 - val_acc: 0.9510


In [41]:
test_dir = '/Users/coviam/image_search/bag.jpg'
image = cv.imread(test_dir)
for index,score in enumerate(model.predict(image.reshape((1,380,380,3)))[0]):
    print label_map[index], score*100

TA-1000023 14.942479133605957
TO-1000007 8.46768543124199
BA-1000026 0.1577880117110908
AT-1000050 0.055035389959812164
BA-1000375 2.0861515775322914
PA-1000016 0.3171001560986042
SE-1000025 3.7588663399219513
WE-1000004 1.6010681167244911
JA-1000033 3.8456622511148453
JA-1000110 7.6397716999053955
AK-1000074 4.525816813111305
PE-1000168 2.8074676170945168
SA-1000009 24.301600456237793
SN-1000003 0.9326579980552197
PA-1000050 0.29640975408256054
BA-1000043 0.31215238850563765
PE-1000538 2.8460783883929253
KA-1000201 1.5435811132192612
DO-1000006 1.2606171891093254
PA-1000010 1.7788020893931389
AT-1000001 0.49057891592383385
MU-1000007 1.087071094661951
AT-1000004 0.367514044046402
FB-1000010 0.05707084201276302
FB-1000014 0.062042276840656996
BA-1000023 0.12995173456147313
HE-1000004 4.734040796756744
DR-1000004 1.4048516750335693
PE-1000104 0.31164197716861963
PA-1000047 0.1992535311728716
OU-1000002 0.13596671633422375
SL-1000004 0.4688163287937641
KA-1000044 0.08292417624033988
KA-1

In [14]:
model.save('models/catgegory_class.h5')
with open('models/cat_label.json', 'w') as f:
    json.dump(label_map, f)